## Analyze one of the datasets from the 2013 SMLM challenge with 3D-DAOSTORM

In this example, we're analyzing a simulated dataset that is 2D, so we'll use the 2D fixed PSF model in 3D-DAOSTORM.

### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import os

os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

### Download the dataset from the 2013 SMLM challenge website

The dataset we'll use is [here](http://bigwww.epfl.ch/smlm/challenge2013/datasets/Bundled_Tubes_High_Density/index.html). This is the "Bundled Tubes High Density" dataset.

You'll need to download the "sequence.zip" file to the above directory and the unzip it. Look for the "Download all frames" button.

### Create a tif movie from the tif images

In [ ]:
import glob
import tifffile

tif_files = glob.glob(os.path.join("sequence", "*.tif"))

with tifffile.TiffWriter("movie.tif") as tf:
    for fname in sorted(tif_files):
        image = tifffile.imread(fname)
        tf.save(image)


### Create an XML file for analysis

In [ ]:
import storm_analysis.sa_library.parameters as parameters

params = parameters.ParametersDAO()

# Analyze the whole movie.
params.setAttr("max_frame", "int", -1)    
params.setAttr("start_frame", "int", -1)

params.setAttr("background_sigma", "float", 8.0)

# These were specified on the website for this dataset.
params.setAttr("camera_gain", "float", 1.0)
params.setAttr("camera_offset", "float", 100.0)

params.setAttr("find_max_radius", "int", 5)
params.setAttr("foreground_sigma", "float", 1.0)
params.setAttr("iterations", "int", 20)
params.setAttr("model", "string", "2dfixed")
params.setAttr("pixel_size", "float", 100.0)
params.setAttr("roi_size", "int", 9)

# Convert from FWHM in nanometers to sigma in pixels.
params.setAttr("sigma", "float", 258.21/(100.0 * 2.355))
params.setAttr("threshold", "float", 6)

# Don't do tracking.
params.setAttr("radius", "float", "0")
params.setAttr("descriptor", "string", "1")

# Don't do drift-correction.
params.setAttr("drift_correction", "int", 0)

params.toXMLFile("analysis.xml", pretty = True)

### Analyze the data

In [ ]:
import storm_analysis.daostorm_3d.mufit_analysis as mfit

# Delete any stale results.
if os.path.exists("movie.hdf5"):
    os.remove("movie.hdf5")
    
%time mfit.analyze("movie.tif", "movie.hdf5", "analysis.xml")

Reference results:

```
Added 46444
   4708 peak finding iterations.

   0 fits reset due to Cholesky failure.
   14 fits reset due to image margin.
   677 fits reset due to negative value in fit function.
   115 fits reset due to negative height.
   534072 fits reset due to non-decreasing error (LM).
   0 fits did not converge.
   17 fits were lost.
   415 peaks lost to proximity filter.
   244 peaks lost to low significance.
   6711697 fitting iterations.

Tracking.

Checking z values.

Analysis complete
CPU times: user 55 s, sys: 171 ms, total: 55.1 s
Wall time: 55 s
```

### Check analysis of a single frame

In [ ]:
import storm_analysis.jupyter_examples.overlay_image as overlay_image
overlay_image.overlayImage("movie.tif", "movie.hdf5", 5)

### Create an image of the analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot

import storm_analysis.sa_utilities.hdf5_to_image as h5_image

sr_im = h5_image.render2DImage("movie.hdf5", scale = 4, sigma = 1)

pyplot.figure(figsize = (8, 8))
pyplot.imshow(sr_im, cmap = "gray")
pyplot.title("SR Image")
pyplot.show()
